## 1 Caratteristiche del Dataset

- I prezzi base variano tra 1.000 e 3.000 (unità monetaria)
- I volumi base variano tra 100 e 500 unità
- Include effetti stagionali (±20% basato sul mese)
- Incorpora elasticità dei prezzi: quando i prezzi aumentano (5-15%), i volumi tendono a diminuire
- Aggiunge rumore casuale per simulare variabilità di mercato


Struttura:
- Per ogni settimana, ci sono 6 osservazioni (3 rotte × 2 dimensioni di container)
- Totale osservazioni attese: 52 settimane × 3 rotte × 2 dimensioni = 312 righe

Questo dataset simula in modo realistico le dinamiche del mercato dei container marittimi, includendo:

- Variazioni stagionali
- Relazione prezzo-domanda
- Differenziazione per rotte e dimensioni
- Fluttuazioni casuali del mercato

In [17]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# 1. Generazione dati fittizi
def generate_sample_data(n_weeks=52):
    dates = pd.date_range(start='2023-01-01', periods=n_weeks, freq='W')
    
    # Creiamo dati per diverse rotte commerciali
    routes = ['ASIA-EUR', 'EUR-USA', 'ASIA-USA']
    container_sizes = ['20ft', '40ft']
    
    data = []
    
    for route in routes:
        for size in container_sizes:
            # Base price variabile per rotta e dimensione container
            base_price = np.random.uniform(1000, 3000)
            base_volume = np.random.uniform(100, 500)
            
            for date in dates:
                # Aggiungiamo seasonality
                seasonal_factor = 1 + 0.2 * np.sin(2 * np.pi * date.month / 12)
                
                # Aggiungiamo random noise
                price = base_price * seasonal_factor * (1 + np.random.uniform(-0.1, 0.1))
                volume = base_volume * seasonal_factor * (1 + np.random.uniform(-0.2, 0.2))
                
                # Price elasticity effect
                if np.random.random() > 0.5:
                    price_increase = np.random.uniform(0.05, 0.15)
                    price *= (1 + price_increase)
                    volume *= (1 - price_increase * 0.5)  # volume decreases with price increase
                
                data.append({
                    'date': date,
                    'route': route,
                    'container_size': size,
                    'price': round(price, 2),
                    'volume': round(volume, 0),
                    'revenue': round(price * volume, 2)
                })
    
    return pd.DataFrame(data)

In [18]:
# Generiamo i dati
print("Generating sample data...")
df = generate_sample_data()
df

Generating sample data...


,date,route,container_size,price,volume,revenue
0,2023-01-01,ASIA-EUR,20ft,2166.91,206.0,446493.89
1,2023-01-08,ASIA-EUR,20ft,2190.92,192.0,419948.49
2,2023-01-15,ASIA-EUR,20ft,2346.49,236.0,553221.32
3,2023-01-22,ASIA-EUR,20ft,2403.14,217.0,520405.51
4,2023-01-29,ASIA-EUR,20ft,2501.84,214.0,534816.56
...,...,...,...,...,...,...
307,2023-11-26,ASIA-USA,40ft,1179.41,334.0,393662.04
308,2023-12-03,ASIA-USA,40ft,1585.89,390.0,618174.82
309,2023-12-10,ASIA-USA,40ft,1495.68,345.0,515594.82
310,2023-12-17,ASIA-USA,40ft,1318.91,398.0,524464.07


## 2 Previsione Volumi:

Usa Random Forest per prevedere i volumi
Include feature engineering per date e variabili categoriche
Calcola la precisione del modello (MAPE)

In [19]:
# 2. Modello di previsione volumi
class VolumePredictor:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        
    def prepare_features(self, df):
        df = df.copy()
        df['month'] = df['date'].dt.month
        df['week'] = df['date'].dt.isocalendar().week
        # One-hot encoding per variabili categoriche
        features = pd.get_dummies(df[['month', 'week', 'route', 'container_size', 'price']])
        return features
        
    def fit(self, df):
        features = self.prepare_features(df)
        self.model.fit(features, df['volume'])
        
    def predict(self, df):
        features = self.prepare_features(df)
        return self.model.predict(features)

In [20]:
# Creiamo e addestriamo il modello di previsione
print("\nTraining volume prediction model...")
predictor = VolumePredictor()
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
predictor.fit(train_df)

# Facciamo previsioni
print("\nMaking predictions...")
test_predictions = predictor.predict(test_df)
mape = np.mean(np.abs((test_df['volume'] - test_predictions) / test_df['volume'])) * 100
print(f"Model MAPE: {mape:.2f}%")


Training volume prediction model...

Making predictions...
Model MAPE: 12.08%


In [21]:
# 3. Analisi Price Elasticity
def calculate_price_elasticity(df):
    elasticities = {}
    
    for route in df['route'].unique():
        for size in df['container_size'].unique():
            mask = (df['route'] == route) & (df['container_size'] == size)
            route_data = df[mask]
            # Calcolo elasticità come % cambio volume / % cambio prezzo
            price_pct_change = route_data['price'].pct_change()
            volume_pct_change = route_data['volume'].pct_change()
            # Rimuoviamo infiniti e NaN
            valid_mask = ~(np.isinf(volume_pct_change/price_pct_change)) & ~np.isnan(volume_pct_change/price_pct_change)
            elasticity = (volume_pct_change[valid_mask]/price_pct_change[valid_mask]).mean()
            elasticities[(route, size)] = round(elasticity, 3)
    return elasticities

## L'elasticità
L'elasticità della domanda al prezzo è un concetto economico fondamentale che misura quanto la domanda (in questo caso il volume delle spedizioni) risponde alle variazioni di prezzo. Analizziamo i risultati:

1. **Interpretazione generale**:
- Un'elasticità negativa significa che quando il prezzo aumenta, il volume diminuisce (relazione inversa, come ci si aspetta)
- Un'elasticità di -1 significa che una variazione dell'1% del prezzo causa una variazione dell'1% del volume in direzione opposta
- Elasticità < -1: domanda elastica (molto sensibile al prezzo)
- Elasticità tra -1 e 0: domanda anelastica (poco sensibile al prezzo)
- Elasticità positiva (raro): potrebbe indicare un bene di lusso o anomalie nei dati


In [22]:
# Calcoliamo le elasticità
print("\nCalculating price elasticities...")
elasticities = calculate_price_elasticity(df)
for (route, size), elasticity in elasticities.items():
    print(f"Route: {route}, Size: {size}, Elasticity: {elasticity}")


Calculating price elasticities...
Route: ASIA-EUR, Size: 20ft, Elasticity: 1.982
Route: ASIA-EUR, Size: 40ft, Elasticity: -0.482
Route: EUR-USA, Size: 20ft, Elasticity: 0.21
Route: EUR-USA, Size: 40ft, Elasticity: -3.402
Route: ASIA-USA, Size: 20ft, Elasticity: -0.079
Route: ASIA-USA, Size: 40ft, Elasticity: 3.48


2. **Analisi specifica**:

a) **ASIA-EUR**:
- 20ft: -6.08 → Molto elastica. Un aumento dell'1% del prezzo causa una diminuzione del 6.08% del volume
- 40ft: -1.51 → Elastica, ma meno della 20ft. La domanda è più stabile

b) **EUR-USA**:
- 20ft: -2.234 → Elastica. Sensibile alle variazioni di prezzo
- 40ft: -127.621 → Questo valore estremamente alto potrebbe indicare:
  * Anomalie nei dati
  * Un mercato estremamente competitivo
  * Potrebbe richiedere ulteriori indagini

c) **ASIA-USA**:
- 20ft: -0.536 → Anelastica. La domanda è relativamente stabile rispetto alle variazioni di prezzo
- 40ft: 0.489 → Positiva (caso insolito). Potrebbe indicare:
  * Un segmento di mercato premium
  * Possibili problemi nei dati
  * Fattori esterni che influenzano la domanda

3. **Implicazioni pratiche per il pricing**:

- Per rotte con alta elasticità (come EUR-USA 40ft), aumentare i prezzi potrebbe essere rischioso e causare perdite significative di volume
- Per rotte con bassa elasticità (come ASIA-USA 20ft), c'è più flessibilità nel pricing
- La rotta ASIA-USA 40ft richiede un'analisi più approfondita dato il valore positivo insolito

In [25]:
import ipywidgets as widgets
from IPython.display import display, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class PriceSimulator:
    def __init__(self, base_data):
        self.df = base_data
        self.elasticities = self.calculate_elasticities()
        self.setup_widgets()
        self.setup_initial_plot()
        
    def calculate_elasticities(self):
        elasticities = {}
        for route in self.df['route'].unique():
            for size in self.df['container_size'].unique():
                mask = (self.df['route'] == route) & (self.df['container_size'] == size)
                route_data = self.df[mask].copy()
                
                # Ordiniamo i dati per prezzo per calcolare l'elasticità
                route_data = route_data.sort_values('price')
                
                # Calcoliamo le variazioni percentuali
                price_pct_change = route_data['price'].pct_change()
                volume_pct_change = route_data['volume'].pct_change()
                
                # Calcoliamo l'elasticità come media delle elasticità punto per punto
                valid_mask = (abs(price_pct_change) > 0.001) & ~np.isinf(volume_pct_change/price_pct_change)
                if valid_mask.sum() > 0:
                    point_elasticities = (volume_pct_change[valid_mask] / price_pct_change[valid_mask])
                    # Filtriamo valori estremi
                    point_elasticities = point_elasticities[abs(point_elasticities) < 10]
                    elasticity = point_elasticities.median()
                else:
                    elasticity = -1.0  # Default elasticity
                
                # Forziamo l'elasticità ad essere negativa (legge della domanda)
                elasticity = min(elasticity, -0.3)
                elasticities[(route, size)] = round(elasticity, 3)
                
        return elasticities
    
    def setup_widgets(self):
        self.route_selector = widgets.Dropdown(
            options=sorted(self.df['route'].unique()),
            description='Route:',
            style={'description_width': 'initial'}
        )
        
        self.size_selector = widgets.Dropdown(
            options=sorted(self.df['container_size'].unique()),
            description='Container:',
            style={'description_width': 'initial'}
        )
        
        self.price_slider = widgets.FloatSlider(
            value=0,
            min=-50,
            max=50,
            step=1,
            description='Price Change %:',
            style={'description_width': 'initial'}
        )
        
        self.metrics_output = widgets.Output()
        
        self.container = widgets.VBox([
            widgets.HBox([self.route_selector, self.size_selector]),
            self.price_slider,
            self.metrics_output
        ])
        
        self.route_selector.observe(self.update_display, 'value')
        self.size_selector.observe(self.update_display, 'value')
        self.price_slider.observe(self.update_display, 'value')
    
    def setup_initial_plot(self):
        self.fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=('Price Impact', 'Volume Impact', 'Revenue Impact'),
            specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]]
        )
        
        self.fig.add_trace(
            go.Bar(x=['Base', 'New'], y=[0, 0],
                  marker_color=['rgb(173,216,230)', 'rgb(0,0,139)']),
            row=1, col=1
        )
        
        self.fig.add_trace(
            go.Bar(x=['Base', 'New'], y=[0, 0],
                  marker_color=['rgb(144,238,144)', 'rgb(0,100,0)']),
            row=1, col=2
        )
        
        self.fig.add_trace(
            go.Bar(x=['Base', 'New'], y=[0, 0],
                  marker_color=['rgb(255,182,193)', 'rgb(139,0,0)']),
            row=1, col=3
        )
        
        self.fig.update_layout(
            height=400,
            width=1000,
            showlegend=False,
            title_text="Price Impact Analysis",
            title_x=0.5
        )
        
        self.fig.update_yaxes(tickformat="$,.0f", title_text="Price", row=1, col=1)
        self.fig.update_yaxes(tickformat=",.0f", title_text="Volume", row=1, col=2)
        self.fig.update_yaxes(tickformat="$,.0f", title_text="Revenue", row=1, col=3)
        
        self.graph_widget = go.FigureWidget(self.fig)
    
    def calculate_impact(self):
        route = self.route_selector.value
        size = self.size_selector.value
        price_change_pct = self.price_slider.value / 100
        
        mask = (self.df['route'] == route) & (self.df['container_size'] == size)
        base_data = self.df[mask].iloc[-1]
        
        base_price = base_data['price']
        base_volume = base_data['volume']
        elasticity = self.elasticities[(route, size)]
        
        # Calcolo nuovo prezzo
        new_price = base_price * (1 + price_change_pct)
        
        # Calcolo variazione volume usando l'elasticità
        volume_change_pct = elasticity * price_change_pct
        # Limitiamo la variazione del volume a ±50%
        volume_change_pct = np.clip(volume_change_pct, -0.5, 0.5)
        new_volume = base_volume * (1 + volume_change_pct)
        
        # Calcolo ricavi
        base_revenue = base_price * base_volume
        new_revenue = new_price * new_volume
        revenue_change_pct = ((new_revenue / base_revenue) - 1) * 100
        
        return {
            'base_price': base_price,
            'new_price': new_price,
            'base_volume': base_volume,
            'new_volume': new_volume,
            'base_revenue': base_revenue,
            'new_revenue': new_revenue,
            'revenue_change_pct': revenue_change_pct,
            'elasticity': elasticity
        }
    
    def update_display(self, change):
        results = self.calculate_impact()
        
        # Aggiorniamo il grafico
        with self.graph_widget.batch_update():
            # Aggiorniamo i titoli
            self.graph_widget.layout.title.text = f"Impact Analysis for {self.route_selector.value} - {self.size_selector.value} (Elasticity: {results['elasticity']:.2f})"
            
            self.graph_widget.layout.annotations[0].text = f'Price (${results["base_price"]:.0f} → ${results["new_price"]:.0f})'
            self.graph_widget.layout.annotations[1].text = f'Volume ({results["base_volume"]:.0f} → {results["new_volume"]:.0f})'
            self.graph_widget.layout.annotations[2].text = f'Revenue (${results["base_revenue"]:,.0f} → ${results["new_revenue"]:,.0f})'
            
            # Aggiorniamo i dati
            self.graph_widget.data[0].y = [results['base_price'], results['new_price']]
            self.graph_widget.data[1].y = [results['base_volume'], results['new_volume']]
            self.graph_widget.data[2].y = [results['base_revenue'], results['new_revenue']]
        
        # Aggiorniamo i metrics
        with self.metrics_output:
            self.metrics_output.clear_output()
            print(f"\nElasticity for {self.route_selector.value} - {self.size_selector.value}: {results['elasticity']:.3f}")
            print(f"\nPrice: ${results['base_price']:.2f} → ${results['new_price']:.2f}")
            print(f"Volume: {results['base_volume']:.0f} → {results['new_volume']:.0f}")
            print(f"Revenue: ${results['base_revenue']:,.2f} → ${results['new_revenue']:,.2f}")
            print(f"Revenue Change: {results['revenue_change_pct']:.1f}%")
    
    def display_simulator(self):
        display(self.container)
        display(self.graph_widget)
        self.update_display(None)

# Generazione dati e avvio simulatore
df = generate_sample_data()
simulator = PriceSimulator(df)
simulator.display_simulator()

FigureWidget({
    'data': [{'marker': {'color': ['rgb(173,216,230)', 'rgb(0,0,139)']},
              'type': 'bar',
              'uid': '2200d2ca-6c20-43b5-8fc3-4bc6aef2684a',
              'x': [Base, New],
              'xaxis': 'x',
              'y': [0, 0],
              'yaxis': 'y'},
             {'marker': {'color': ['rgb(144,238,144)', 'rgb(0,100,0)']},
              'type': 'bar',
              'uid': '687a5f56-f347-45b5-8402-df04f6f1c50f',
              'x': [Base, New],
              'xaxis': 'x2',
              'y': [0, 0],
              'yaxis': 'y2'},
             {'marker': {'color': ['rgb(255,182,193)', 'rgb(139,0,0)']},
              'type': 'bar',
              'uid': 'b03d335c-c073-4a79-ae31-bd8f0762a207',
              'x': [Base, New],
              'xaxis': 'x3',
              'y': [0, 0],
              'yaxis': 'y3'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text

### Elasticità -0.3 if price increase by 10%:
È un'elasticità anelastica (|ε| < 1)
Significa che il volume cambia proporzionalmente meno del prezzo
Questo è tipico per beni/servizi necessari o con poche alternative


### Effetto sul business:
Prezzo +10% → Volume -3% (per via dell'elasticità -0.3)
Il ricavo aumenta (+6.7%) perché l'aumento del prezzo compensa la perdita di volume
Questo suggerisce che aumentare i prezzi su questa rotta potrebbe essere profittevole


### Interpretazione pratica:
L'elasticità -0.3 indica che i clienti sono relativamente insensibili alle variazioni di prezzo
Un aumento del 10% del prezzo causa solo una perdita del 3% del volume
Questo potrebbe indicare:

In [28]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def generate_sample_data(n_weeks=52):
    dates = pd.date_range(start='2023-01-01', periods=n_weeks, freq='W')
    routes = ['ASIA-EUR', 'EUR-USA', 'ASIA-USA']
    container_sizes = ['20ft', '40ft']
    
    data = []
    np.random.seed(42)  # Per riproducibilità
    
    for route in routes:
        for size in container_sizes:
            base_price = np.random.uniform(1000, 3000)
            base_volume = np.random.uniform(100, 500)
            
            for date in dates:
                seasonal_factor = 1 + 0.2 * np.sin(2 * np.pi * date.month / 12)
                # Aggiungiamo più variabilità ai prezzi per calcolare meglio l'elasticità
                price = base_price * seasonal_factor * (1 + np.random.uniform(-0.2, 0.2))
                # Volume inversamente correlato al prezzo
                price_effect = -0.3 * (price - base_price) / base_price
                volume = base_volume * seasonal_factor * (1 + price_effect + np.random.uniform(-0.1, 0.1))
                
                data.append({
                    'date': date,
                    'route': route,
                    'container_size': size,
                    'price': round(price, 2),
                    'volume': round(volume, 0),
                    'revenue': round(price * volume, 2)
                })
    
    return pd.DataFrame(data)

class PriceSimulator:
    def __init__(self, base_data):
        self.df = base_data
        self.elasticities = self.calculate_elasticities()
        self.setup_widgets()
        self.setup_plots()
        
    def calculate_elasticities(self):
        elasticities = {}
        for route in self.df['route'].unique():
            for size in self.df['container_size'].unique():
                mask = (self.df['route'] == route) & (self.df['container_size'] == size)
                route_data = self.df[mask].copy()
                
                # Calcoliamo le variazioni percentuali
                price_pct_change = route_data['price'].pct_change()
                volume_pct_change = route_data['volume'].pct_change()
                
                # Filtriamo i valori validi
                valid_mask = (abs(price_pct_change) > 0.001) & ~np.isinf(volume_pct_change/price_pct_change)
                if valid_mask.sum() > 0:
                    point_elasticities = volume_pct_change[valid_mask] / price_pct_change[valid_mask]
                    # Filtriamo valori estremi
                    point_elasticities = point_elasticities[abs(point_elasticities) < 10]
                    elasticity = point_elasticities.median()
                else:
                    elasticity = -0.3  # Default elasticity
                
                # Forziamo l'elasticità ad essere negativa e ragionevole
                elasticity = np.clip(elasticity, -2.0, -0.1)
                elasticities[(route, size)] = round(elasticity, 3)
                
        return elasticities
    
    def setup_widgets(self):
        self.route_selector = widgets.Dropdown(
            options=sorted(self.df['route'].unique()),
            description='Route:',
            style={'description_width': 'initial'}
        )
        
        self.size_selector = widgets.Dropdown(
            options=sorted(self.df['container_size'].unique()),
            description='Container:',
            style={'description_width': 'initial'}
        )
        
        self.price_slider = widgets.FloatSlider(
            value=0,
            min=-50,
            max=50,
            step=1,
            description='Price Change %:',
            style={'description_width': 'initial'}
        )
        
        self.elasticity_slider = widgets.FloatSlider(
            value=-0.3,
            min=-2.0,
            max=-0.1,
            step=0.1,
            description='Test Elasticity:',
            style={'description_width': 'initial'}
        )
        
        self.metrics_output = widgets.Output()
        self.optimization_output = widgets.Output()
        
        self.tab = widgets.Tab()
        self.tab.children = [
            widgets.VBox([
                widgets.HBox([self.route_selector, self.size_selector]),
                self.price_slider,
                self.metrics_output
            ]),
            widgets.VBox([
                self.elasticity_slider,
                self.optimization_output
            ])
        ]
        self.tab.set_title(0, 'Price Simulator')
        self.tab.set_title(1, 'Optimization Analysis')
        
        self.route_selector.observe(self.update_all, 'value')
        self.size_selector.observe(self.update_all, 'value')
        self.price_slider.observe(self.update_display, 'value')
        self.elasticity_slider.observe(self.update_optimization, 'value')
    
    def setup_plots(self):
        self.fig_main = make_subplots(
            rows=1, cols=3,
            subplot_titles=('Price Impact', 'Volume Impact', 'Revenue Impact')
        )
        
        self.fig_main.add_trace(
            go.Bar(x=['Base', 'New'], y=[0, 0],
                  marker_color=['rgb(173,216,230)', 'rgb(0,0,139)']),
            row=1, col=1
        )
        
        self.fig_main.add_trace(
            go.Bar(x=['Base', 'New'], y=[0, 0],
                  marker_color=['rgb(144,238,144)', 'rgb(0,100,0)']),
            row=1, col=2
        )
        
        self.fig_main.add_trace(
            go.Bar(x=['Base', 'New'], y=[0, 0],
                  marker_color=['rgb(255,182,193)', 'rgb(139,0,0)']),
            row=1, col=3
        )
        
        self.fig_main.update_layout(
            height=400,
            width=1000,
            showlegend=False,
            title_text="Price Impact Analysis",
            title_x=0.5
        )
        
        self.fig_main.update_yaxes(tickformat="$,.0f", title_text="Price", row=1, col=1)
        self.fig_main.update_yaxes(tickformat=",.0f", title_text="Volume", row=1, col=2)
        self.fig_main.update_yaxes(tickformat="$,.0f", title_text="Revenue", row=1, col=3)
        
        self.graph_widget_main = go.FigureWidget(self.fig_main)
        
        # Plot per la curva di domanda e ricavi
        self.fig_demand = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Demand Curve', 'Revenue Curve')
        )
        
        self.fig_demand.add_trace(
            go.Scatter(x=[], y=[], mode='lines', name='Demand'),
            row=1, col=1
        )
        
        self.fig_demand.add_trace(
            go.Scatter(x=[], y=[], mode='lines', name='Revenue'),
            row=1, col=2
        )
        
        self.fig_demand.update_layout(
            height=400,
            width=1000,
            showlegend=True,
            title_text="Demand and Revenue Analysis",
            title_x=0.5
        )
        
        self.fig_demand.update_xaxes(title_text="Price ($)", row=1, col=1)
        self.fig_demand.update_xaxes(title_text="Price ($)", row=1, col=2)
        self.fig_demand.update_yaxes(title_text="Volume", row=1, col=1)
        self.fig_demand.update_yaxes(title_text="Revenue ($)", row=1, col=2)
        
        self.graph_widget_demand = go.FigureWidget(self.fig_demand)
    
    def calculate_impact(self):
        route = self.route_selector.value
        size = self.size_selector.value
        price_change_pct = self.price_slider.value / 100
        
        mask = (self.df['route'] == route) & (self.df['container_size'] == size)
        base_data = self.df[mask].iloc[-1]
        
        base_price = base_data['price']
        base_volume = base_data['volume']
        elasticity = self.elasticities[(route, size)]
        
        new_price = base_price * (1 + price_change_pct)
        volume_change_pct = elasticity * price_change_pct
        volume_change_pct = np.clip(volume_change_pct, -0.5, 0.5)
        new_volume = base_volume * (1 + volume_change_pct)
        
        base_revenue = base_price * base_volume
        new_revenue = new_price * new_volume
        revenue_change_pct = ((new_revenue / base_revenue) - 1) * 100
        
        return {
            'base_price': base_price,
            'new_price': new_price,
            'base_volume': base_volume,
            'new_volume': new_volume,
            'base_revenue': base_revenue,
            'new_revenue': new_revenue,
            'revenue_change_pct': revenue_change_pct,
            'elasticity': elasticity
        }
    
    def calculate_optimal_price(self, base_price, base_volume, elasticity):
        if elasticity >= -1:
            return base_price * 1.5
        else:
            optimal_markup = -1 / elasticity
            return base_price * (1 + optimal_markup)
    
    def generate_demand_curve(self, base_price, base_volume, elasticity, points=50):
        price_range = np.linspace(base_price * 0.5, base_price * 1.5, points)
        volumes = []
        revenues = []
        
        for p in price_range:
            price_change = (p - base_price) / base_price
            volume_change = elasticity * price_change
            volume = base_volume * (1 + volume_change)
            volumes.append(volume)
            revenues.append(p * volume)
            
        return price_range, volumes, revenues
    
    def update_demand_curves(self, results):
        prices, volumes, revenues = self.generate_demand_curve(
            results['base_price'],
            results['base_volume'],
            results['elasticity']
        )
        
        with self.graph_widget_demand.batch_update():
            self.graph_widget_demand.data[0].x = prices
            self.graph_widget_demand.data[0].y = volumes
            self.graph_widget_demand.data[1].x = prices
            self.graph_widget_demand.data[1].y = revenues
            
            # Rimuoviamo i punti correnti se esistono
            if len(self.graph_widget_demand.data) > 2:
                self.graph_widget_demand.data = self.graph_widget_demand.data[:2]
            
            # Aggiungiamo i nuovi punti correnti
            self.graph_widget_demand.add_trace(
                go.Scatter(x=[results['new_price']], 
                          y=[results['new_volume']], 
                          mode='markers',
                          marker=dict(size=10, color='red'),
                          name='Current Point'),
                row=1, col=1
            )
            
            self.graph_widget_demand.add_trace(
                go.Scatter(x=[results['new_price']], 
                          y=[results['new_revenue']], 
                          mode='markers',
                          marker=dict(size=10, color='red'),
                          name='Current Revenue'),
                row=1, col=2
            )
    
    def update_display(self, change):
        results = self.calculate_impact()
        
        with self.graph_widget_main.batch_update():
            self.graph_widget_main.layout.title.text = f"Impact Analysis for {self.route_selector.value} - {self.size_selector.value} (Elasticity: {results['elasticity']:.2f})"
            
            self.graph_widget_main.layout.annotations[0].text = f'Price (${results["base_price"]:.0f} → ${results["new_price"]:.0f})'
            self.graph_widget_main.layout.annotations[1].text = f'Volume ({results["base_volume"]:.0f} → {results["new_volume"]:.0f})'
            self.graph_widget_main.layout.annotations[2].text = f'Revenue (${results["base_revenue"]:,.0f} → ${results["new_revenue"]:,.0f})'
            
            self.graph_widget_main.data[0].y = [results['base_price'], results['new_price']]
            self.graph_widget_main.data[1].y = [results['base_volume'], results['new_volume']]
            self.graph_widget_main.data[2].y = [results['base_revenue'], results['new_revenue']]
        
        self.update_demand_curves(results)
        
        with self.metrics_output:
            self.metrics_output.clear_output()
            print(f"\nElasticity for {self.route_selector.value} - {self.size_selector.value}: {results['elasticity']:.3f}")
            print(f"\nPrice: ${results['base_price']:.2f} → ${results['new_price']:.2f}")
            print(f"Volume: {results['base_volume']:.0f} → {results['new_volume']:.0f}")
            print(f"Revenue: ${results['base_revenue']:,.2f} → ${results['new_revenue']:,.2f}")
            print(f"Revenue Change: {results['revenue_change_pct']:.1f}%")
    
    def update_optimization(self, change):
        with self.optimization_output:
            self.optimization_output.clear_output()
            
            route = self.route_selector.value
            size = self.size_selector.value
            mask = (self.df['route'] == route) & (self.df['container_size'] == size)
            base_data = self.df[mask].iloc[-1]
            
            test_elasticity = self.elasticity_slider.value
            optimal_price = self.calculate_optimal_price(
                base_data['price'], 
                base_data['volume'],
                test_elasticity
            )
            
            print(f"\nOptimization Analysis for {route} - {size}")
            print(f"Test Elasticity: {test_elasticity:.2f}")
            print(f"\nCurrent Price: ${base_data['price']:.2f}")
            print(f"Optimal Price: ${optimal_price:.2f}")
            print(f"Suggested Change: {((optimal_price/base_data['price'])-1)*100:.1f}%")
            
            # Calcola l'impatto del prezzo ottimale
            optimal_change = (optimal_price - base_data['price']) / base_data['price']
            optimal_volume = base_data['volume'] * (1 + test_elasticity * optimal_change)
            optimal_revenue = optimal_price * optimal_volume
            
            current_revenue = base_data['price'] * base_data['volume']
            revenue_improvement = ((optimal_revenue/current_revenue)-1)*100
            
            print(f"\nProjected Results at Optimal Price:")
            print(f"Volume Change: {((optimal_volume/base_data['volume'])-1)*100:.1f}%")
            print(f"Revenue Improvement: {revenue_improvement:.1f}%")
            
            # Aggiorna i grafici con la nuova elasticità
            test_results = {
                'base_price': base_data['price'],
                'new_price': optimal_price,
                'base_volume': base_data['volume'],
                'new_volume': optimal_volume,
                'base_revenue': current_revenue,
                'new_revenue': optimal_revenue,
                'elasticity': test_elasticity
            }
            
            self.update_demand_curves(test_results)
        
    def update_all(self, change):
        self.update_display(change)
        self.update_optimization(change)
    
    def display_simulator(self):
        display(self.tab)
        display(self.graph_widget_main)
        display(self.graph_widget_demand)
        self.update_all(None)

# Generazione dati e avvio simulatore
if __name__ == "__main__":
    df = generate_sample_data()
    simulator = PriceSimulator(df)
    simulator.display_simulator()


FigureWidget({
    'data': [{'marker': {'color': ['rgb(173,216,230)', 'rgb(0,0,139)']},
              'type': 'bar',
              'uid': 'cba11614-aef0-4543-8ae8-8ea43a47dd46',
              'x': [Base, New],
              'xaxis': 'x',
              'y': [0, 0],
              'yaxis': 'y'},
             {'marker': {'color': ['rgb(144,238,144)', 'rgb(0,100,0)']},
              'type': 'bar',
              'uid': '5fd9ff9a-6c0b-46bf-a112-437cc955a642',
              'x': [Base, New],
              'xaxis': 'x2',
              'y': [0, 0],
              'yaxis': 'y2'},
             {'marker': {'color': ['rgb(255,182,193)', 'rgb(139,0,0)']},
              'type': 'bar',
              'uid': 'a593a2e5-6bb8-4fb8-bfce-862f6b11b6dd',
              'x': [Base, New],
              'xaxis': 'x3',
              'y': [0, 0],
              'yaxis': 'y3'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Demand',
              'type': 'scatter',
              'uid': '1f71d21b-df17-4144-a0d3-99950fb001d6',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'mode': 'lines',
              'name': 'Revenue',
              'type': 'scatter',
              'uid': 'ffe65790-ae09-4182-9500-6897bf70b1d1',
              'x': [],
              'xaxis': 'x2',
              'y': [],
              'yaxis': 'y2'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Demand Curve',
                                'x': 0.225,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
        